# Output of the symbolic tendencies: Land-Atmosphere model example 

In this notebook, we show how to create the symbolic tendencies of the model. Symbolic tendencies here means that it is possible to make any parameter of the model appears in the tendencies equations.

This can be done in several languages (Python, Julia, Fortran), but here, we are going to use Python.

More details about the model used in this notebook can be found in the articles:
* Hamilton, O., Demaeyer, J., Vannitsem, S., & Crucifix, M. (2025). *Using Unstable Periodic Orbits to Understand Blocking Behaviour in a Low Order Land-Atmosphere Model*. Submitted to Chaos. [preprint](https://doi.org/10.48550/arXiv.2503.02808)
* Xavier, A. K., Demaeyer, J., & Vannitsem, S. (2024). *Variability and predictability of a reduced-order land–atmosphere coupled model.* Earth System Dynamics, **15**(4), 893-912. [doi:10.5194/esd-15-893-2024](https://doi.org/10.5194/esd-15-893-2024)

or in the documentation.

## Modules import

First, setting the path and loading of some modules

In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from numba import njit

In [ ]:
from scipy.integrate import solve_ivp

Importing the model's modules

In [ ]:
from qgs.params.params import QgParams

In [ ]:
from qgs.functions.symbolic_tendencies import create_symbolic_equations
from qgs.functions.tendencies import create_tendencies

## Systems definition

General parameters

In [ ]:
# Time parameters
dt = 0.1


Setting some model parameters

In [ ]:
model_parameters = QgParams({'phi0_npi': np.deg2rad(50.)/np.pi, 'n':1.3 }, dynamic_T=False)

and defining the spectral modes used by the model (they must be *symbolic*)

In [ ]:
# Mode truncation at the wavenumber 2 in both x and y spatial coordinate for the atmosphere
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Same modes for the ground temperature modes
model_parameters.set_ground_channel_fourier_modes(2, 2, mode="symbolic")

Completing the model parameters

In [ ]:
# Changing (increasing) the orography depth
model_parameters.ground_params.set_orography(0.2, 1)
# Setting the parameters of the heat transfer from the soil
model_parameters.gotemperature_params.set_params({'gamma': 1.6e7, 'T0': 300})
model_parameters.atemperature_params.set_params({ 'hlambda':10, 'T0': 290})
# Setting atmospheric parameters
model_parameters.atmospheric_params.set_params({'sigma': 0.2, 'kd': 0.085, 'kdp': 0.02})

# Setting insolation 
model_parameters.gotemperature_params.set_params({})

In [ ]:
C_g = 300
model_parameters.atemperature_params.set_insolation(0.4*C_g , 0)

model_parameters.gotemperature_params.set_insolation(C_g , 0)

In [ ]:
# Printing the model's parameters
model_parameters.print_params()

## Outputting the model equations

Calculating the tendencies in Python as a function of the parameters $C_{{\rm o},0}$, $C_{{\rm a},0}$, $k_d$ and $k'_d$:

In [ ]:
%%time
funcs, = create_symbolic_equations(model_parameters, continuation_variables=[model_parameters.gotemperature_params.C[0], model_parameters.atemperature_params.C[0], model_parameters.atmospheric_params.kd, model_parameters.atmospheric_params.kdp], language='python')

Let's inspect the output:

In [ ]:
print(funcs)

Note that the tendencies have been already formatted as a [Numba](https://numba.pydata.org/) function, but it is easy to extract the equations for any other kind of accelerator or simply to produce pure Python code.

It is now easy to get the function into operation:

In [ ]:
exec(funcs)

and

In [ ]:
f(0.,np.zeros(model_parameters.ndim), 300., 120., 0.085, 0.02)

## Comparing with numerical results

We can check that the symbolic (parameters dependent) equations are the same as the `qgs` numerical ones (with the same values of the parameters):

In [ ]:
f_num, Df = create_tendencies(model_parameters)

In [ ]:
f_num(0., np.zeros(model_parameters.ndim))

In addition, one can easily compare the obtained attractors:

In [ ]:
# IC calculated from a long transient
ic = np.array([0.05055959, -0.01639403, -0.01440781, -0.01846523, -0.01352099,
        0.011685  , -0.00201673, -0.02030682,  0.03923588, -0.02229535,
        0.0586372 , -0.01805569, -0.01264252, -0.0103574 , -0.00618456,
        0.01159318, -0.00478694, -0.00782509,  0.01066059, -0.01552667,
        0.30718325, -0.03247899, -0.04512935, -0.00078786, -0.00067468,
        0.00183836,  0.00068025,  0.00215424, -0.00322845, -0.00186392])

# Actual integration
traj = solve_ivp(f, (0., 100000.), ic, t_eval=np.arange(0, 100000., dt), args=(300., 120., 0.085, 0.02))
traj_num = solve_ivp(f_num, (0., 100000.), ic, t_eval=np.arange(0, 100000., dt))

In [ ]:
varx = 2
vary = 1
plt.figure(figsize=(12, 10))

plt.plot(traj.y[varx], traj.y[vary], marker='o', ms=0.03, ls='', label='Symbolic tendencies')
plt.plot(traj_num.y[varx], traj_num.y[vary], marker='o', ms=0.03, ls='', label='Fully numerical tendencies')


plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');
# plt.legend()

Fully numerical tendencies attractor is in orange while the symbolic tendencies on is in blue

## Varying the parameters

The obvious possibilities given by the symbolic tendencies are to allow users to easily perform sensitivity analysis:

In [ ]:
# let's start with 4 different values of the friction k_d
kd_list = [0.06, 0.085, 0.095, 0.1, 0.105, 0.12]

# let's compute the attractor for each

attractor_list = list()

for kd in kd_list:
    attractor_list.append(solve_ivp(f, (0., 100000.), ic, t_eval=np.arange(0, 100000., dt), args=(300., 120., kd, 0.02)))

In [ ]:
varx = 2
vary = 1
plt.figure(figsize=(18, 6))

k=1
for kd, attractor in zip(kd_list, attractor_list):
    plt.subplot(2, 3, k)
    plt.plot(attractor.y[varx], attractor.y[vary], marker='o', ms=0.03, ls='', label=f'$k_d$ = {kd}')
    plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
    plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

    plt.legend()
    k+=1



